In [1]:
from dotenv import load_dotenv
from urllib import parse
import numpy as np
import pandas as pd
import requests
import time
import json
import csv
import os

load_dotenv()
api_key = os.getenv('api_key') #api key 불러오기

request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

base_url = "https://kr.api.riotgames.com/tft/"

In [2]:
'''
(1) summonerid : 유저 id code, String이다.

(2) summonerName : 게임에서 사용하고 있는 유저의 id, String이다.

(3) leaguePoints : 현재 rank 점수, int이다.

(4) rank : rank. 챌린저이므로 모든 유저가 'I'.

(5) wins : 이번 rank 측정 기간 중 승리 수, int이다.

(6) losses : 이번 rank 측정 기간 중 패배 수, int이다.

(7) veteran : 베테랑 유무. (이번 시즌 게임 수가 100회 이상이면 베테랑), Boolean이다.

(8) inactive : 최근 게임 참여 적극성, Boolean이다.

(9) freshBlood : 이번 시즌이 첫 시즌인 경우 T, Boolean이다.

(10) hotStreak : 3연승 이상 중, Boolean이다.
'''
challenger_url = base_url + 'league/v1/challenger'

challenger = requests.get(challenger_url, headers=request_header).json()

challenger_df = pd.DataFrame(challenger['entries'])

challenger_df.to_csv('test.csv')

print(challenger_df)

                                            summonerId  \
0     GYpEe07LBmd7GybNpSoe2-fgdYMSzVmMagNxg4ciENaw6sDT   
1      quWqFkt--TijmVjtjxEGffcnV6JnBrUnnXTjRB5vIypA87Y   
2    K4MlDPgSz-CVS8bO-6I1IOGoTP8MNGROw82mH5ikDY0Cr2...   
3      sAPLpDlRwfjbGV2ZDghc0Cs2LzJIxqxszHZtrD-uEptTCq4   
4      Vm_hgLbRFnovCtF6yrBFvYNO9EySsS4ni1L4y7kvSp6ymv8   
..                                                 ...   
295  bhUFFgKMnQS28IOePlaFRqp2pDCCwj-WelqWR3_5VlJZ5z...   
296  jw0kVj0vIJkW0cQA-XEHGpD_SqPsxtTC4VKjU8kYaEe1FU...   
297  -J_7sCyLnQPCRBUxJIYlacz_7VPseHUIHP4bFWROJFwInZ...   
298  BtdzIruPUodO5rtV3R8w5x721CzkWIA3aTMdWOb2t_wZWY...   
299  oHWRTjxl2GFP9LXMFBSVb4wHCWMPM32ytFDbpvsVByWNvF...   

                                                 puuid  leaguePoints rank  \
0    4up9RrUC7olP4EwQfrAK27cSAX-SdTybr8KVkn5TG9n_60...          1795    I   
1    3eDtpya9tImZkaBw_CRlfMORSvVdIGyXSfCV9W7KfdJ0wu...          1759    I   
2    x09y6MrvUVvcQlW2mri3OrJUXdn1-6FQSLl18H76t-zh6c...          1612    

In [3]:
# challenger tier url
challenger_url = base_url + 'league/v1/challenger'

# request 함수 사용하여 정보 불러오기, 헤더 정보는 설정한 request_header 사용
challenger = requests.get(challenger_url, headers=request_header).json()

# pandas 이용하여 dataframe 생성
challenger_df = pd.DataFrame(challenger['entries'])

# challenger_puuid_list 이름의 list 생성
challenger_puuid_list = []

# challenger_df의 puuid만 추출하여 list로 변환
challenger_puuid_list = challenger_df['puuid'].tolist()

# 위의 코드들이 잘 실행되었는지 확인
# print(challenger_puuid_list)

In [ ]:
'''
- 수정사항
    함수명 변경 fetch_summoner_matchID -> fetch_challenger_matchID
    함수명 변경 fetch_all_summoner_matchIDs -> fetch_all_challenger_matchIDs
    fetch_all_summoner_matchIDs 함수의 불필요한 header 삭제
    url 변경
    api_key 변경

puuid를 이용해 user별 matchID(20개) 추출, 비동기 코드 사용
'''
import aiohttp
import asyncio
from asyncio_throttle import Throttler  # asyncio_throttle 가져오기

# 비동기적으로 소환사 matchID를 가져오는 함수
async def fetch_challenger_matchID(session, puuid, header, throttler):
    url = f'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=20'
    
    async with throttler:  # 요청 속도를 제한하는 Throttler 사용
        async with session.get(url, headers=header) as response:
            if response.status == 429:  # Rate limit 처리
                retry_after = int(response.headers.get('Retry-After', 60))
                print(f"Rate limit exceeded for {puuid}. Sleeping for {retry_after} seconds.")
                await asyncio.sleep(retry_after)
                return await fetch_challenger_matchID(session, puuid, header, throttler)  # 재시도
            
            if response.status == 200:
                summoner_match_data = await response.json()
                return {'puuid': puuid, 'match_ids': summoner_match_data}
            else:
                print(f"Error: {response.status} for PUUID {puuid}")
                return {'puuid': puuid, 'match_ids': None}

# 비동기적으로 여러 소환사의 matchID를 가져오는 함수
async def fetch_all_challenger_matchIDs(api_key, summoner_puuid_list):
    summoner_matchID_list = []
    throttler = Throttler(rate_limit=20, period=1)  # 초당 최대 20개의 요청만 허용

    async with aiohttp.ClientSession() as session:
        tasks = []
        for puuid in summoner_puuid_list:
            task = fetch_challenger_matchID(session, puuid, request_header, throttler)  # 비동기 작업 생성
            tasks.append(task)
        
        # 모든 비동기 작업을 병렬로 실행
        responses = await asyncio.gather(*tasks)
        
        for response in responses:
            if response and response.get('match_ids') is not None:
                summoner_matchID_list.append(response)

    return summoner_matchID_list

# 비동기 실행을 위한 메인 함수
async def get_all_challenger_matchIDs():
    # 여러 소환사의 matchID를 비동기로 가져오기
    summoner_matchID_list = await fetch_all_challenger_matchIDs(api_key, challenger_puuid_list)
    
    # 결과 반환
    return summoner_matchID_list

# 비동기 함수 실행
challenger_matchID_list = await get_all_challenger_matchIDs()

# 결과 출력
print(f"Challenger MatchID List: {challenger_matchID_list}")

In [41]:
# ex2_match_ids = challenger_matchID_list[0:25]
# print(ex2_match_ids)
# print(type(challenger_matchID_list))

# for match_ids in challenger_matchID_list:
#     print(match_ids)
#     challenger_matchIDs = match_ids['match_ids']

print(len(challenger_matchID_list))
challenger_match_ids = []
for i in challenger_matchID_list:
    challenger_match_ids.append(i['match_ids'])

print(challenger_match_ids)

300
[['KR_7569313788', 'KR_7569093505', 'KR_7564817015', 'KR_7564787094', 'KR_7564647977', 'KR_7563713698', 'KR_7563693566', 'KR_7563670273', 'KR_7563643696', 'KR_7563608599', 'KR_7563589643', 'KR_7563569374', 'KR_7563553791', 'KR_7563549253', 'KR_7563543101', 'KR_7562950748', 'KR_7562885352', 'KR_7562833483', 'KR_7562768682', 'KR_7562708396'], ['KR_7565607807', 'KR_7563472607', 'KR_7563456716', 'KR_7563437415', 'KR_7563408810', 'KR_7563386069', 'KR_7562019969', 'KR_7559807836', 'KR_7558553499', 'KR_7558524721', 'KR_7558499793', 'KR_7558479291', 'KR_7558457343', 'KR_7558440343', 'KR_7558417804', 'KR_7558249618', 'KR_7558224936', 'KR_7558205092', 'KR_7558169765', 'KR_7558124568'], ['KR_7575152190', 'KR_7575106488', 'KR_7573212140', 'KR_7569998541', 'KR_7569087964', 'KR_7569050290', 'KR_7565477170', 'KR_7564490157', 'KR_7564462396', 'KR_7562740002', 'KR_7561438028', 'KR_7561317872', 'KR_7561224180', 'KR_7559911223', 'KR_7559846107', 'KR_7559792037', 'KR_7559742248', 'KR_7559707614', 'KR_

In [42]:
'''
- 수정사항
    header 삭제 후 수정 (header -> request_header)
    api_key 삭제
    url 수정
'''

# 비동기적으로 match 데이터를 가져오는 함수
async def fetch_match_data(session, match_id, api_key, throttler):
    url = f'https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}'
    
    async with throttler:  # 요청 속도 제한
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:  # Rate limit 처리
                retry_after = int(response.headers.get('Retry-After', 60))  # Retry-After 헤더 확인
                print(f"Rate limit exceeded for match {match_id}. Sleeping for {retry_after} seconds.")
                await asyncio.sleep(retry_after)  # 비동기 대기
                return await fetch_match_data(session, match_id, api_key, throttler)  # 다시 시도
            elif response.status == 200:
                return await response.json()  # JSON 데이터 반환
            else:
                print(f"Error: {response.status} for match ID {match_id}")
                return None

# 여러 match_id에 대해 비동기적으로 데이터를 가져오는 함수
async def fetch_all_matches(challenger_match_ids, api_key):
    throttler = Throttler(rate_limit=20, period=1)  # 초당 20개의 요청으로 제한
    async with aiohttp.ClientSession() as session:
        tasks = []
        for match_id in challenger_match_ids:
            task = fetch_match_data(session, match_id, api_key, throttler)
            tasks.append(task)
        
        responses = await asyncio.gather(*tasks)

        # 유효한 데이터만 수집 (None 제외)
        valid_responses = [response for response in responses if response is not None]
        return valid_responses

# 비동기 메인 함수
async def main():
    # 모든 match_ids에 대해 게임 정보 가져오기
    match_data_list = await fetch_all_matches(challenger_match_ids, api_key)
    
    # 결과 출력 및 반환
    print(f"Total matches fetched: {len(match_data_list)}")
    return match_data_list


# Jupyter 환경에서는 asyncio.run() 대신 await main() 사용
fetched_matches = await main()

# 여기서 결과를 리스트에 저장
print(f"Fetched Matches List: {fetched_matches}")

Error: 404 for match ID ['KR_7561964942', 'KR_7561917558', 'KR_7561863306', 'KR_7561804074', 'KR_7561740112', 'KR_7561578466', 'KR_7561495359', 'KR_7561428193', 'KR_7559286864', 'KR_7559270888', 'KR_7559250499', 'KR_7558316337', 'KR_7558307479', 'KR_7558297848', 'KR_7558294398', 'KR_7558280958', 'KR_7558258698', 'KR_7558207413', 'KR_7557270755', 'KR_7557246975']
Error: 404 for match ID ['KR_7569379864', 'KR_7569332614', 'KR_7569279425', 'KR_7569234138', 'KR_7569099187', 'KR_7561826535', 'KR_7561750730', 'KR_7561676711', 'KR_7561396242', 'KR_7561310152', 'KR_7561067131', 'KR_7561000664', 'KR_7560942574', 'KR_7560480649', 'KR_7560458753', 'KR_7560437518', 'KR_7560406525', 'KR_7559707614', 'KR_7559675118', 'KR_7559634493']
Error: 404 for match ID ['KR_7574773594', 'KR_7571409826', 'KR_7564566338', 'KR_7562289625', 'KR_7561462627', 'KR_7559088524', 'KR_7559029502', 'KR_7558770504', 'KR_7556296124', 'KR_7556281151', 'KR_7555483450', 'KR_7555468299', 'KR_7555354625', 'KR_7554721019', 'KR_755